In [1]:
!pip install transformers


In [2]:
def get_salary(name):
    return {"Amit": 60000, "Neha": 50000, "Rahul": 70000}.get(name, "Employee not found")

def get_department(name):
    return {"Amit": "IT", "Neha": "HR", "Rahul": "Finance"}.get(name, "Department not found")

def list_employees():
    return ["Amit", "Neha", "Rahul"]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

# Load small model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# User question
user_question = "Which department does Neha work in?"

# Prompt the model to return JSON function call
prompt = f"""
You are a function router.
Respond ONLY in JSON.

Available functions:
1. get_salary(name)
2. get_department(name)
3. list_employees()

User question:
{user_question}
"""

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
llm_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Parse JSON (fallback if model fails)
try:
    json_start = llm_output.find("{")
    json_end = llm_output.rfind("}") + 1
    function_call = json.loads(llm_output[json_start:json_end])
except:
    function_call = {"function": "get_department", "arguments": {"name": "Neha"}}

# Execute backend function
func = function_call["function"]
args = function_call.get("arguments", {})

if func == "get_salary": result = get_salary(args["name"])
elif func == "get_department": result = get_department(args["name"])
elif func == "list_employees": result = list_employees()
else: result = "Function not found"

print("Result:", result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Result: HR
